# Example of how to use Divvy

Start by importing `divvy`, and then create a new `ComputingConfiguration` object. Normally, you'd pass your own `DIVCFG` file in as an argument to the constructor, and you will get your own computing packages. But with no arguments, we'll just get a few default packages:

In [1]:
import divvy
dcc = divvy.ComputingConfiguration()

No local config file was provided
No global config file was provided in environment variable DIVCFG
Using default config file.
Loading divvy config file: /home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/default_compute_settings.yaml
Available packages: set(['default', 'slurm', 'local'])
Activating compute package 'default'


This loads up the default compute package, and we see that there are a few other packages available. We can explore the compute settings in the loaded (`default`) package like this: 

In [2]:
dcc.compute

{'submission_command': 'sh', 'submission_template': '/home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/localhost_template.sub'}

That gives us a list of key-value pairs that make up this compute package. We can activate a different one like this: 

In [3]:
dcc.activate_package("slurm")

Activating compute package 'slurm'


True

It returns 'True' to indicate that the activation has been successful. This will change our settings. Let's inspect the new package:

In [4]:
dcc.compute

{'submission_command': 'sbatch', 'example_variable': 'blah', 'submission_template': '/home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/slurm_template.sub'}

Now that we've activated the package of interest, we can use those variables to populate a submission template to create a submission script. Use the `write_script` function:

In [5]:
s = dcc.write_script("test_script.sub")

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


This function opens the template specified by the `submission_template` variable in the compute package, and then populates any template variables with values from the compute package. In this case, our compute package didn't have any relevant variables and so the template is returned without change:

In [6]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}



In addition to using the compute package variables, we can also pass some extra variables along to the the `write_script` function. Let's pass a value for the `code` variable and see how this changes the submission script output:

In [7]:
s = dcc.write_script("test_script.sub", {"code":"yellow"})

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


Here's the output. Notice that the `{CODE}` variable has been replaced with the word `yellow`:

In [8]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

yellow



## Using a priority list of variables

Now, you can also pass more than one `Dict` object, in priority order, by just passing a list. Here, we'll pass 2 dicts, and any values in the 2nd one will override values in the first:

In [9]:
s = dcc.write_script("test_script.sub", [{"code":"yellow", "time": "now"}, {"code": "red"}])

Writing script to /home/nsheff/code/divvy/docs_jupyter/test_script.sub


In [10]:
with open("test_script.sub") as f:
    print(f.read())

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='now'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

red

